In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt

## filepath

In [2]:
train_data = "../Switchboard-Corpus/swda_data/train_set.txt"
test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
val_data = "../Switchboard-Corpus/swda_data/val_set.txt"

In [18]:
train_dataset = []
f = open(train_data, "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [19]:
len(train_dataset)

192390

In [20]:
test_dataset = []
f = open(test_data, "r", encoding='utf-8')
for row in f:
    test_dataset.append(row.strip())
f.close()

In [21]:
len(test_dataset)

4078

In [22]:
val_dataset = []
f = open(val_data, "r", encoding='utf-8')
for row in f:
    val_dataset.append(row.strip())
f.close()

In [23]:
len(val_dataset)

3272